In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model
import os
import io

In [5]:
import pandas as pd
import numpy as np
import os
import io

# --- 1. Function to Extract and Clean 'High Drought' Data from soilmoisture.csv ---
def extract_high_drought_data(filepath):
    """
    Parses the messy soilmoisture.csv file and extracts only the data 
    associated with the '# high drougt' section, keeping ONLY the Ambient Temp 
    and Object Temp (the second and third columns). The Soil Moisture column is dropped.
    """
    print(f"Parsing: {filepath}")
    
    start_marker = "# high drougt"
    end_marker = "# medium high dought" 
    
    data_lines = []
    is_in_section = False
    
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip()
                
                if line.startswith(start_marker):
                    is_in_section = True
                    continue
                
                if is_in_section and line.startswith(end_marker):
                    break
                
                if is_in_section:
                    # Ignore comment lines and non-numeric lines
                    if line and not line.startswith('#') and not any(char.isalpha() for char in line):
                        try:
                            # Split by comma. Expected structure: Soil Moisture, Ambient Temp, Object Temp.
                            parts = [float(p.strip()) for p in line.split(',')]
                            if len(parts) == 3:
                                # KEEP Ambient Temp (parts[1]) and Object Temp (parts[2])
                                data_lines.append([parts[1], parts[2]]) 
                        except ValueError:
                            continue

        if not data_lines:
            raise ValueError("No valid data found in the 'high drougt' section.")
            
        # Create DataFrame with only the desired temperature columns
        high_drought_df = pd.DataFrame(data_lines, columns=['Drought_AmbientTemp', 'Drought_ObjectTemp'])
        print(f"Successfully extracted {len(high_drought_df)} high drought temperature rows.")
        return high_drought_df

    except FileNotFoundError:
        print(f"Error: The file {filepath} was not found.")
        return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred during parsing: {e}")
        return pd.DataFrame()

# --- 2. Main Processing Logic ---
def process_data_and_combine():
    # File paths
    soil_moisture_filepath = 'soilmoisture.csv'
    rgb_filepath = '30to40.csv'
    
    # Base name for the output file with sequential naming logic
    base_name = 'combined_rgb_temp_data'
    extension = '.csv'
    output_filepath = base_name + extension
    counter = 1
    
    # Sequential Naming: combined_rgb_temp_data.csv, combined_rgb_temp_data_1.csv, etc.
    while os.path.exists(output_filepath):
        print(f"File {output_filepath} already exists. Generating new filename.")
        output_filepath = f"{base_name}_{counter}{extension}"
        counter += 1

    # A. Extract the high drought data (Ambient Temp and Object Temp only)
    drought_df = extract_high_drought_data(soil_moisture_filepath)
    
    if drought_df.empty:
        print("Aborting combination process due to missing or empty drought data.")
        return

    # B. Load the RGB data
    print(f"Loading RGB data from: {rgb_filepath}")
    try:
        rgb_df = pd.read_csv(rgb_filepath)
        # Clean RGB columns by removing potential extra whitespace from column names
        rgb_df.columns = rgb_df.columns.str.strip()
        
        # Ensure only the R, G, B columns are kept from this file. 
        # We assume the first file has columns like Index, Row, Col, R, G, B
        if all(col in rgb_df.columns for col in ['R', 'G', 'B']):
            # Keep R, G, B and drop any index/row/col metadata
            rgb_df = rgb_df[['R', 'G', 'B']]
        else:
            print("Warning: R, G, or B columns not found in 30to40.csv. Keeping all columns from RGB file.")

        print(f"Successfully loaded {len(rgb_df)} RGB rows.")
    except Exception as e:
        print(f"Error loading RGB file: {e}")
        return

    # C. Combine the datasets (Cycling the smaller temperature data over the larger RGB data)
    num_rgb_rows = len(rgb_df)
    
    # Repeat the two-column temperature data and trim it to the exact length of the RGB data
    num_drought_rows = len(drought_df)
    num_repeats = int(np.ceil(num_rgb_rows / num_drought_rows))
    repeated_drought_df = pd.concat([drought_df] * num_repeats, ignore_index=True).head(num_rgb_rows)
    
    # Ensure indices are aligned for concatenation
    rgb_df = rgb_df.reset_index(drop=True)
    
    # Combine the two dataframes side-by-side (column-wise)
    combined_df = pd.concat([rgb_df, repeated_drought_df], axis=1)
    
    print(f"Data combination complete. Final dataset size: {len(combined_df)} rows.")

    # D. Save the result to the safe output path
    combined_df.to_csv(output_filepath, index=False)
    print(f"\n--- SUCCESS ---")
    print(f"Clean, combined data saved to: {output_filepath}")
    print(f"Final columns: {list(combined_df.columns)}")

if __name__ == "__main__":
    process_data_and_combine()


Parsing: soilmoisture.csv
Successfully extracted 2 high drought temperature rows.
Loading RGB data from: 30to40.csv
Successfully loaded 10000 RGB rows.
Data combination complete. Final dataset size: 10000 rows.

--- SUCCESS ---
Clean, combined data saved to: combined_rgb_temp_data.csv
Final columns: ['R', 'G', 'B', 'Drought_AmbientTemp', 'Drought_ObjectTemp']


In [8]:
import pandas as pd
import os

# --- Constants for CWSI Calculation ---
# Lower Limit (LL) of the temperature difference (Tobj - Tair) for non-stressed conditions
LOWER_LIMIT = -2.06

# Upper Limit (UL) of the temperature difference (Tobj - Tair) for maximum stress/no-transpiration
UPPER_LIMIT = -1.33

def calculate_cwsi(filepath):
    """
    Reads a CSV file, calculates the CWSI, adds it as a *new* column 
    (CWSI), and saves the modified DataFrame back to the original filepath, 
    preserving all existing columns.

    CWSI Formula: CWSI = (dT - LL) / (UL - LL)
    Where:
        dT is (Object_Temp - Ambient_Temp)
        LL is LOWER_LIMIT (-2.06)
        UL is UPPER_LIMIT (-1.33)
    """
    print(f"Starting CWSI calculation for file: '{filepath}'")
    
    if not os.path.exists(filepath):
        print(f"Error: Input file not found at '{filepath}'")
        return

    try:
        # Load the CSV file into a pandas DataFrame
        df = pd.read_csv(filepath)
        
        # Check if the DataFrame has at least 2 columns
        if df.shape[1] < 2:
            print("Error: The CSV must contain at least two columns for temperature data.")
            return

        # Identify the columns: second-to-last is Ambient, last is Object
        ambient_temp_col_index = df.columns[-2]
        object_temp_col_index = df.columns[-1]
        
        # Print identified columns for user confirmation
        print(f"Identified Ambient Temperature Column: '{ambient_temp_col_index}'")
        print(f"Identified Object Temperature Column: '{object_temp_col_index}'")

        # Ensure the columns are numeric before calculation
        ambient_temps = pd.to_numeric(df[ambient_temp_col_index], errors='coerce')
        object_temps = pd.to_numeric(df[object_temp_col_index], errors='coerce')

        # 1. Calculate the Temperature Difference (Delta T)
        # Delta T = T_Object - T_Ambient
        # This is added as an intermediate column for transparency.
        df['Delta_T'] = object_temps - ambient_temps
        
        # 2. Calculate the CWSI
        # CWSI = (Delta_T - LL) / (UL - LL)
        numerator = df['Delta_T'] - LOWER_LIMIT
        denominator = UPPER_LIMIT - LOWER_LIMIT
        
        # Create the new 'CWSI' column. This does not overwrite any existing data.
        df['CWSI'] = numerator / denominator

        # Round CWSI values and clip between 0 and 1
        df['CWSI'] = df['CWSI'].round(3).clip(lower=0.0, upper=1.0)
        
        # Save the resulting DataFrame back to the original file path.
        # This overwrites the file, but includes all the original data PLUS the new columns.
        df.to_csv(filepath, index=False)
        
        print(f"\nCWSI calculation complete. The file '{filepath}' has been updated.")
        print("A new 'CWSI' column and an intermediate 'Delta_T' column have been appended.")
        
        print("\n--- Updated File Preview (Original Columns + New Data) ---")
        print(df.head())
        
    except Exception as e:
        print(f"An unexpected error occurred during processing: {e}")

# --- Script Execution ---
if __name__ == "__main__":
    # Define the input file path (this file will be updated)
    input_file = 'combined_rgb_temp_data.csv'
    
    # Run the calculation
    calculate_cwsi(input_file)


Starting CWSI calculation for file: 'combined_rgb_temp_data.csv'
Identified Ambient Temperature Column: 'Drought_AmbientTemp'
Identified Object Temperature Column: 'Drought_ObjectTemp'

CWSI calculation complete. The file 'combined_rgb_temp_data.csv' has been updated.
A new 'CWSI' column and an intermediate 'Delta_T' column have been appended.

--- Updated File Preview (Original Columns + New Data) ---
    R    G   B  Drought_AmbientTemp  Drought_ObjectTemp  Delta_T   CWSI
0  80  130  77                26.71               25.37    -1.34  0.986
1  84  131  75                26.71               25.41    -1.30  1.000
2  87  128  80                26.71               25.37    -1.34  0.986
3  93  126  87                26.71               25.41    -1.30  1.000
4  93  127  90                26.71               25.37    -1.34  0.986


In [9]:
from scipy.stats import zscore

def filter_rgb_outliers(input_filepath, output_filepath, zscore_threshold=3):
    """
    Reads a CSV file, calculates the Z-scores for R, G, and B columns,
    and removes any row where the absolute Z-score of R, G, or B is
    greater than the specified threshold (default is 3).

    Args:
        input_filepath (str): The path to the input CSV file.
        output_filepath (str): The path where the filtered data will be saved.
        zscore_threshold (float): The absolute Z-score value above which a
                                  row is considered an outlier and removed.
    """
    print(f"Reading data from: {input_filepath}")

    # Check if the input file exists
    if not os.path.exists(input_filepath):
        print(f"Error: Input file not found at '{input_filepath}'.")
        return

    try:
        # 1. Read the CSV file
        df = pd.read_csv(input_filepath)
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return

    # Define the columns used for outlier detection
    rgb_columns = ['R', 'G', 'B']

    # Ensure required columns are present
    missing_cols = [col for col in rgb_columns if col not in df.columns]
    if missing_cols:
        print(f"Error: CSV is missing required RGB columns: {missing_cols}. Please check column names.")
        return

    # 2. Calculate Z-scores for R, G, and B columns
    # zscore() calculates the Z-score column-wise by default
    z_scores = df[rgb_columns].apply(zscore)

    # 3. Create a boolean mask to identify outliers
    # We mark a row as an outlier (True) if the absolute Z-score of ANY RGB component is > threshold.
    # The .any(axis=1) checks horizontally (across R, G, B) for any True value in the row.
    outlier_mask = (np.abs(z_scores) > zscore_threshold).any(axis=1)

    # Calculate statistics before filtering
    total_rows = len(df)
    outlier_rows = outlier_mask.sum()
    kept_rows = total_rows - outlier_rows

    print(f"\n--- Z-Score Analysis ---")
    print(f"Z-Score Threshold: > {zscore_threshold}")
    print(f"Total rows in input: {total_rows}")
    print(f"Outlier rows found (deleted): {outlier_rows}")

    # 4. Delete/Filter the outlier rows
    # We select rows where the outlier_mask is False (meaning NOT an outlier)
    df_filtered = df[~outlier_mask]

    # 5. Save the results to a new file
    df_filtered.to_csv(output_filepath, index=False)
    print(f"\nFiltered data saved successfully to: {output_filepath}")
    print(f"Rows kept in output file: {kept_rows}")


if __name__ == "__main__":
    # Define file paths
    input_file = 'combined_rgb_temp_data.csv'
    output_file = 'filtered_rgb_data.csv'

    # Run the filtering process
    filter_rgb_outliers(input_file, output_file)

    print("\n\n--- Done ---")
    print("To run this script, ensure you have pandas and numpy/scipy installed:")
    print("pip install pandas numpy scipy")


Reading data from: combined_rgb_temp_data.csv

--- Z-Score Analysis ---
Z-Score Threshold: > 3
Total rows in input: 10000
Outlier rows found (deleted): 26

Filtered data saved successfully to: filtered_rgb_data.csv
Rows kept in output file: 9974


--- Done ---
To run this script, ensure you have pandas and numpy/scipy installed:
pip install pandas numpy scipy
